In [13]:
import torch 
import torch.nn as nn

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transform

[아래는 해당 링크를 따라서 스스로 작성해본 코드](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_3_1_visdom-example.ipynb)

[다양한 사용 예시](https://github.com/facebookresearch/visdom) 

### 단, Visdom을 사용하려면 Visdom 서버가 켜져있어야만 한다!

In [3]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


#### 이제 서버에서 진행을 하게 됨 


아래의 것들을 입력하면 서버에서 Text가 출현하고, Image가 출현하고 ... 

### Text 띄우기

In [4]:
vis.text("Hello, World!", env='main') # env='main: 나중에 한 번에 모든 창을 끌 때, 사용할 수 있음. => 웬만하면 쓰자. 

'window_38d9783f270856'

### Image 띄우기 

In [9]:
a = torch.randn(3, 200, 200)
vis.image(a)

# vis.images(a) 를 하면 여러 개의 이미지를 띄울 수 있음. 

'window_38d97893126fac'

### 여러 개의 image 띄우기

In [11]:
vis.images(torch.Tensor(3,3,28,28))

C:\Users\anaconda\Anaconda3\lib\site-packages\numpy\core\_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


'window_38d979482dafa6'

### MNIST Data & CIFAR- 10 띄우기

In [15]:
MNIST = datasets.MNIST(root='./MNIST_data',
                     train = True,
                     transform = transform.ToTensor(), 
                     download=True)

CIFAR10 = datasets.CIFAR10(root='./cifar10', 
                          train=True,
                          transform = transform.ToTensor(),
                          download=True)

Files already downloaded and verified


### CIFAR10 이미지 가지고 와서 visdom에 띄우기

In [23]:
cifar10 = CIFAR10.__getitem__(0)
print(cifar10)
vis.images(cifar10[0], env='main')

(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.0314,  ..., 0.1961, 0.1961, 0

'window_38d97acd9f1706'

### mnist 데이터 가지고 오기

In [24]:
mnist = MNIST.__getitem__(0)
print(mnist[0].shape)
vis.images(mnist[0], env='main')

torch.Size([1, 28, 28])


'window_38d97b00ca42a2'

### 데이터 한 번에 많이 띄우기

In [25]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST, 
                                         batch_size=32,
                                         shuffle=False)

In [27]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [28]:
vis.close(env='main')

''

---

#### 학습을 진행하다 보면, loss 등에 대해서 graph로 표현할 일들이 많아질 것

### Line Plot

#### X값을 넣지 않으면, X값은 default로 0과 1 사이의 값들로만 구성됨

In [31]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [32]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [33]:
X_append = torch.Tensor([6])
Y_append = torch.randn(1)

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_38d97c42ec84d2'

### 한 윈도우에 여러 개의 라인 그리기

In [36]:
num = torch.Tensor(list(range(0,10)))
num = num.view(-1, 1)
num = torch.cat((num, num), dim=1)

print(num)

tensor([[0., 0.],
        [1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.],
        [8., 8.],
        [9., 9.]])


In [37]:
plt = vis.line(Y=torch.randn(10,2), X=num)

### 선들에 범례 (Legend) 추가하기 

In [39]:
# opts 활용. Dictionary 형태로 넣어주면 된다. showlegend: 축의 이름
plt = vis.line(Y=Y_data, X=X_data opts=dict(title='Test', showlegend=True)) 

In [40]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title='Test', legend=['1번'], showlegend=True))

In [41]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번','2번'], showlegend=True))

### Line Update를 위해 아예 함수로 선언해버리기

CNN에서 간단하게 함수형태로 가져다 써버리려는 의도에서

In [44]:
def loss_tracker(loss_plot, loss_value, num):
    # num과 loss_value는 Tensor 이어야만 한다.
    vis.line(X=num,
            Y=loss_value,
            win = loss_plot, # loss_plot에 업데이트! (아래 코드로 치면 plt라는 플롯에 계속 업데이트를 해달라는 것)
            update = 'append')

In [45]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))